# In-Context Learning (ICL) for Comics

- Initial config: K = 10, N = 1, embedding_model = bert-base-uncased, no attributes/context

### Libraries

In [1]:
import ast
import json
import torch
import random
import pickle
import numpy as np
import pandas as pd
import torch.nn.functional as F

from pathlib import Path
from tqdm.notebook import tqdm
from operator import itemgetter
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

### Tokenizer and Model (embedding and inference)

In [2]:
embedding_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
embedding_model = AutoModel.from_pretrained("google-bert/bert-base-uncased")

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [53]:
# model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"
#model_id = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
# model_id = "unsloth/Meta-Llama-3.1-70B-Instruct-bnb-4bit"
model_id = "unsloth/Qwen2.5-7B-Instruct"


In [64]:
inference_tokenizer = AutoTokenizer.from_pretrained(model_id, padding='left', padding_side='left')
inference_tokenizer.pad_token = inference_tokenizer.eos_token
# #terminators = [
#     inference_tokenizer.eos_token_id,
#     inference_tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

In [55]:
generation_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #cache_dir = '/home/umushtaq/scratch/am_work/in_context_learning/model_downloads',
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/6.88G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/6.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

### Read Data

In [6]:
DATASET_DIR = Path(Path.cwd().as_posix()) / "emotion_analysis_comics" / "incontext_learning" / "datasets"

In [7]:
DATASET_DIR

PosixPath('/Utilisateurs/umushtaq/emotion_analysis_comics/incontext_learning/datasets')

In [8]:
all_labels = ["anger", "surprise", "fear", "disgust", "sadness", "joy", "neutral"]

In [9]:
emotion_map = {
    'AN': 'anger',
    'DI': 'disgust',
    'FE': 'fear',
    'SA': 'sadness',
    'SU': 'surprise',
    'JO': 'joy'
}

In [10]:
df = pd.read_csv(DATASET_DIR / "comics_data_processed.csv")
df = df.drop(columns=[df.columns[0], df.columns[1]]).reset_index(drop=True)

In [11]:
def extract_emotions(row):

    emotion_str = row.emotion

    if emotion_str == 'Neutral':
        return ['neutral']

    emotions = emotion_str.split('-')
    tags = []

    for emotion in emotions:
        abbrev = emotion[:2]  # Get the abbreviation
        value_part = emotion[2:]  # Get the value part
        
        # Ensure that the value part is a valid integer and abbrev is in the emotion_map
        if abbrev in emotion_map and value_part.isdigit():
            value = int(value_part)
            if value > 0:
                tags.append(emotion_map[abbrev].lower())
        else:
            print(f"Warning: Skipping invalid emotion entry: '{emotion}'")
    return tags  

In [12]:
df['emotions_list'] = df.apply(lambda row: extract_emotions(row), axis=1)

In [13]:
df.shape

(5282, 12)

### Get embeddings

In [14]:
utterance_embed_d = {}

for utterance in tqdm(df.utterance):
    # print(utterance)
    while True:
        try:
            inputs = embedding_tokenizer(utterance, return_tensors="pt")
            output = embedding_model(**inputs)
            embedding = output[1][0].squeeze()
            utterance_embed_d[utterance] = embedding.detach().numpy()
            break
        except Exception as e:
            print(e)

  0%|          | 0/5282 [00:00<?, ?it/s]

In [15]:
df['utterance_embedding'] = df.utterance.apply(lambda x: utterance_embed_d[x])

In [16]:
train_df = df[df.split == "TRAIN"].reset_index(drop=True)
test_df = df[df.split == "TEST"].reset_index(drop=True)

### Get K neighbours and prepare prompt

In [17]:
def get_k_neighbours(k, utterance):

    test_utterance_embedding = test_df[test_df.utterance == utterance]["utterance_embedding"].values[0]

    utterance_embed_d = {}
    for e in train_df.iterrows():
        if e[1].utterance not in utterance_embed_d:
            utterance_embed_d[e[1].utterance] = e[1].utterance_embedding

    # train_titles = set(df[df.split == 'TRAIN'].title.unique())
    train_utterances = set(train_df.utterance)

    dist_l = []
    for t, v in utterance_embed_d.items():
        if t in train_utterances:
            # d = cos_sim(title_embed_d[title], v)
            d = F.cosine_similarity(torch.tensor(test_utterance_embedding), torch.tensor(v), dim=0)
            dist_l.append((t, d.item()))

    sorted_dist_l = sorted(dist_l, key=itemgetter(1), reverse=True)
    
    return sorted_dist_l[0: k]

In [18]:
get_k_neighbours(10, test_df.iloc[10]["utterance"])

[('… YES?', 0.9947154521942139),
 ('… POR FAVOR…', 0.9920827150344849),
 ('HEY!', 0.9919130802154541),
 ("NO… NO… YOU CAN'T CONTROL THEM!", 0.9913744330406189),
 ('UM… ACTUALLY…', 0.9905799627304077),
 ('… th-the…', 0.990537703037262),
 ('" DON\'T GO INTO THE LIGHT… "', 0.9902849197387695),
 ('But today…', 0.9900076985359192),
 ("IF HE'D KILLED ANDREA… I DON'T KNOW…", 0.9897928833961487),
 ('QUIET… OH NO…', 0.9897283315658569)]

In [19]:
def prepare_similar_example_prompts(utterance, k=30, seed=33):
    """
    Create a part of prompt made of k examples in the train set, whose topic is most similar to a given title.
    """

    random.seed(seed)

    neighbours_l = get_k_neighbours(2*k, utterance) # Fetch the 2*k closest neighbors
    # print(neighbours_l)
    sampled_neighbours_l = random.sample(neighbours_l, k) # Only keep k of them
    # bprint(sampled_neighbours_l)

    prompt = ''
    cnt = 0
    for i, (utterance, dist) in enumerate(sampled_neighbours_l):
        prompt += f'## Example {i+1}\n'

        example_df = train_df[train_df.utterance == utterance]
        # example_df = example_df[example_df.aty != 'none'].reset_index()
        
        class_l = []
        for k in example_df.iterrows():
            
            # if k[0] == 0:

            #     prompt += f'# Abstract:\n{example_df.iloc[0].utterance}\n\n# Arguments:\n'
            #     cnt = 0
                
            # prompt += f'Argument {cnt + 1}={k[1].text} - Class={k[1].aty}\n'
            prompt += f'Utterance {cnt + 1}={k[1].utterance}\n'
            class_l.append(k[1].emotions_list)
            cnt += 1
            
        prompt += '\n# Result:\n'
        prompt += '{' + ', '.join([f'"utterance_emotions": "{class_l[i]}"' for i in range(len(class_l))]) + '}'
        prompt += '\n\n'

    return prompt

In [20]:
print(prepare_similar_example_prompts(test_df.iloc[10]["utterance"], k=10))

## Example 1
Utterance 1=… LANDED

# Result:
{"utterance_emotions": "['surprise']"}

## Example 2
Utterance 2=… th-the…

# Result:
{"utterance_emotions": "['neutral']"}

## Example 3
Utterance 3=But today…

# Result:
{"utterance_emotions": "['sadness']"}

## Example 4
Utterance 4=IF HE'D KILLED ANDREA… I DON'T KNOW…

# Result:
{"utterance_emotions": "['fear', 'sadness', 'surprise']"}

## Example 5
Utterance 5=SHE… WE COULD STILL…

# Result:
{"utterance_emotions": "['sadness']"}

## Example 6
Utterance 6=BUT FLORIAN… I…

# Result:
{"utterance_emotions": "['fear']"}

## Example 7
Utterance 7=WHAT THE… CAN'T… MOVE MY…

# Result:
{"utterance_emotions": "['fear', 'surprise']"}

## Example 8
Utterance 8=THAT WAS…

# Result:
{"utterance_emotions": "['surprise']"}

## Example 9
Utterance 9=HEY!
Utterance 10=HEY!
Utterance 11=HEY!

# Result:
{"utterance_emotions": "['surprise']", "utterance_emotions": "['anger', 'surprise']", "utterance_emotions": "['anger']"}

## Example 10
Utterance 12=… that

### Prepare test set prompts

In [21]:
experiment_df = test_df

sys_msg_l = []
task_msg_l = []

for row in tqdm(test_df.iterrows(), total=len(test_df)):
    
    #row[0] is index, row[1] is the data
    sys_msg = {"role": "system", "content": "### Task description: You are an expert sentiment analysis assistant that takes an utterance from a comic book and must classify the utterance into appropriate emotion class(s): anger, surprise, fear, disgust, sadness, joy, neutral. You are given one utterance to classify and 3 example utterances to help you. You must absolutely not generate any text or explanation other than the following JSON format: {\"utterance_emotion\": \"<predicted emotion classes for the utterance (str)>}\"\n\n" + "### Examples:\n\n" + prepare_similar_example_prompts(row[1].utterance)}
    #sys_msg = {"role":"system", "content": "### Task description: You are an expert biomedical assistant that takes 1) an abstract text, 2) the list of all arguments from this abstract text, and must classify all arguments into one of two classes: Claim or Premise. " + proportion_desc + " You must absolutely not generate any text or explanation other than the following JSON format {\"Argument 1\": <predicted class for Argument 1 (str)>, ..., \"Argument n\": <predicted class for Argument n (str)>}\n\n### Class definitions:" + " Claim = " + claim_fulldesc + " Premise = " + premise_fulldesc + "\n\n### Examples:\n\n" + prepare_similar_example_prompts(title_l[i], experiment_df, k=3, seed=seed)}  # Sample by similar title
    task_msg = {"role":"user", "content": f"# Utterance:\n{row[1].utterance}\n\n# Result:\n"}
    
    sys_msg_l.append(sys_msg)
    task_msg_l.append(task_msg)
    

  0%|          | 0/1776 [00:00<?, ?it/s]

In [22]:
len(sys_msg_l)

1776

In [23]:
print(sys_msg_l[10]['content'])

### Task description: You are an expert sentiment analysis assistant that takes an utterance from a comic book and must classify the utterance into appropriate emotion class(s): anger, surprise, fear, disgust, sadness, joy, neutral. You are given one utterance to classify and 3 example utterances to help you. You must absolutely not generate any text or explanation other than the following JSON format: {"utterance_emotion": "<predicted emotion classes for the utterance (str)>}"

### Examples:

## Example 1
Utterance 1=MUCH AS I HATE TO INTERRUPT…

# Result:
{"utterance_emotions": "['surprise', 'joy']"}

## Example 2
Utterance 2=BUT FLORIAN… I…

# Result:
{"utterance_emotions": "['fear']"}

## Example 3
Utterance 3=YOUR… MASTER?

# Result:
{"utterance_emotions": "['surprise']"}

## Example 4
Utterance 4=… that someday…

# Result:
{"utterance_emotions": "['sadness']"}

## Example 5
Utterance 5=STOP!

# Result:
{"utterance_emotions": "['anger', 'fear']"}

## Example 6
Utterance 6=THAT'S G

In [24]:
print(task_msg_l[10]["content"])

# Utterance:
@… IN A FAR OFF KINGDOM…

# Result:



In [25]:
prepared_sys_task_msg_l = []

for i in range(len(sys_msg_l)):
    prepared_sys_task_msg_l.append([sys_msg_l[i], task_msg_l[i]])

In [26]:
len(prepared_sys_task_msg_l)

1776

In [27]:
with open(DATASET_DIR / 'tmp_prepared_sys_task_msg_l.pkl', 'wb') as f:
    
    pickle.dump(prepared_sys_task_msg_l, f)

### Run Inferences

In [56]:
# Load the pickled list
with open(DATASET_DIR / 'tmp_prepared_sys_task_msg_l.pkl', 'rb') as f:
    
    prepared_sys_task_msg_l = pickle.load(f)


In [57]:
inputs = inference_tokenizer.apply_chat_template(
            prepared_sys_task_msg_l,
            #tools=tools,
            # pad_token = tokenizer.eos_token,
            padding=True,
            truncation=True,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt",
)

In [58]:
inputs

{'input_ids': tensor([[151645, 151645, 151645,  ..., 151644,  77091,    198],
        [151645, 151645, 151645,  ..., 151644,  77091,    198],
        [151645, 151645, 151645,  ..., 151644,  77091,    198],
        ...,
        [151645, 151645, 151645,  ..., 151644,  77091,    198],
        [151645, 151645, 151645,  ..., 151644,  77091,    198],
        [151645, 151645, 151645,  ..., 151644,  77091,    198]]), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]])}

In [59]:
def batch_tensor(tensor, batch_size):
    return [tensor[i:i+batch_size] for i in range(0, tensor.size(0), batch_size)]

In [60]:
BATCH_SIZE = 32

In [61]:
input_ids_batches = batch_tensor(inputs['input_ids'], BATCH_SIZE)
attention_mask_batches = batch_tensor(inputs['attention_mask'], BATCH_SIZE)

In [62]:
CUDA_LAUNCH_BLOCKING=0

In [65]:
generated_outputs = []

for i, (input_ids_batch, attention_mask_batch) in tqdm(enumerate(zip(input_ids_batches, attention_mask_batches)), total=len(input_ids_batches)):
    
    print(f"Processing batch {i + 1}")
    
    inputs = {
        'input_ids': input_ids_batch.to(generation_model.device),
        'attention_mask': attention_mask_batch.to(generation_model.device)
    }

    outputs = generation_model.generate(
    **inputs,
    max_new_tokens=64,
    pad_token_id=inference_tokenizer.eos_token_id,
    #eos_token_id=terminators,
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
    )
    #outputs = [inference_tokenizer.decode(output[input_ids.shape[-1]:], skip_special_tokens=True) for output in outputs]
    generated_outputs.append(outputs)
    # del inputs


  0%|          | 0/56 [00:00<?, ?it/s]

Processing batch 1
Processing batch 2
Processing batch 3
Processing batch 4
Processing batch 5
Processing batch 6
Processing batch 7
Processing batch 8
Processing batch 9
Processing batch 10
Processing batch 11
Processing batch 12
Processing batch 13
Processing batch 14
Processing batch 15
Processing batch 16
Processing batch 17
Processing batch 18
Processing batch 19
Processing batch 20
Processing batch 21
Processing batch 22
Processing batch 23
Processing batch 24
Processing batch 25
Processing batch 26
Processing batch 27
Processing batch 28
Processing batch 29
Processing batch 30
Processing batch 31
Processing batch 32
Processing batch 33
Processing batch 34
Processing batch 35
Processing batch 36
Processing batch 37
Processing batch 38
Processing batch 39
Processing batch 40
Processing batch 41
Processing batch 42
Processing batch 43
Processing batch 44
Processing batch 45
Processing batch 46
Processing batch 47
Processing batch 48
Processing batch 49
Processing batch 50
Processin

In [66]:
print(len(generated_outputs))

56


In [67]:
inputs['input_ids'].shape[1]

2443

In [68]:
decoded_outputs = []

for batch in generated_outputs:

    for prediction in batch:
        # print(prediction)
        #print(prediction.shape)
        decoded_outputs.append(inference_tokenizer.decode(prediction[inputs['input_ids'].shape[1]:], skip_special_tokens=True))
        #break
        # decoded_outputs.append(inference_tokenizer.decode(prediction, skip_special_tokens=True))
        #inference_tokenizer.decode(prediction[0][inputs_ids.shape[-1]:], skip_special_tokens=True)
        #inference_tokenizer.decode(prediction.shape[[-1]:], skip_special_tokens=True)

In [69]:
len(decoded_outputs)

1776

In [70]:
decoded_outputs

['{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'joy\']"}',
 '{"utterance_emotions": "[\'joy\']"}',
 '{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'joy\', \'surprise\']"}',
 '{"utterance_emotions": "[\'joy\']"}',
 '{"utterance_emotions": "[\'fear\', \'surprise\']"}',
 '{"utterance_emotions": "[\'joy\']"}',
 '{"utterance_emotions": "[\'joy\']"}',
 '{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'neutral\', \'surprise\']"}',
 '{"utterance_emotions": "[\'anger\', \'disgust\']"}',
 '{"utterance_emotions": "[\'anger\']"}',
 '{"utterance_emotions": "[\'anger\', \'surprise\']"}',
 '{"utterance_emotions": "[\'anger\', \'disgust\']"}',
 '{"utterance_emotions": "[\'joy\']"}',
 '{"utterance_emotions": "[\'anger\', \'surprise\', \'joy\']"}',
 '{"utterance_emotions": "[\'anger\']"}',
 '{"utterance_emotions": "[\'anger\', \'disgust\']"}',
 '{"utterance_emotions": "[\'disgust\', \'sadness\']"}',
 '{"utterance_emotions": "[\'anger

In [71]:
preds = []

#for output in outputs_l:
for i, prediction in enumerate(decoded_outputs):
    try:
        # Use json.loads to safely parse the JSON-like string
        parsed_prediction = json.loads(prediction)
        # Append the values of the parsed prediction to preds
        preds.append(parsed_prediction['utterance_emotions'])
        
    except json.JSONDecodeError as e:
        print(f"Error decoding prediction: {i}")

In [72]:
len(preds)

1776

In [73]:
preds

["['neutral']",
 "['joy']",
 "['joy']",
 "['neutral']",
 "['joy', 'surprise']",
 "['joy']",
 "['fear', 'surprise']",
 "['joy']",
 "['joy']",
 "['neutral']",
 "['neutral', 'surprise']",
 "['anger', 'disgust']",
 "['anger']",
 "['anger', 'surprise']",
 "['anger', 'disgust']",
 "['joy']",
 "['anger', 'surprise', 'joy']",
 "['anger']",
 "['anger', 'disgust']",
 "['disgust', 'sadness']",
 "['anger']",
 "['fear', 'surprise']",
 "['surprise']",
 "['joy']",
 "['anger', 'surprise']",
 "['joy']",
 "['joy']",
 "['joy']",
 "['anger', 'disgust']",
 "['anger', 'disgust']",
 "['anger', 'surprise']",
 "['anger', 'surprise']",
 "['joy']",
 "['anger', 'joy']",
 "['anger', 'fear']",
 "['surprise']",
 "['anger', 'disgust']",
 "['sadness', 'respect']",
 "['fear', 'sadness']",
 "['neutral']",
 "['neutral']",
 "['disgust', 'sadness']",
 "['surprise', 'anger']",
 "['anger', 'surprise']",
 "['disgust']",
 "['anger']",
 "['disgust', 'fear']",
 "['anger', 'disgust']",
 "['anger', 'sadness']",
 "['joy']",
 "['neu

In [74]:
preds = [ast.literal_eval(item) for item in preds]

In [75]:
preds

[['neutral'],
 ['joy'],
 ['joy'],
 ['neutral'],
 ['joy', 'surprise'],
 ['joy'],
 ['fear', 'surprise'],
 ['joy'],
 ['joy'],
 ['neutral'],
 ['neutral', 'surprise'],
 ['anger', 'disgust'],
 ['anger'],
 ['anger', 'surprise'],
 ['anger', 'disgust'],
 ['joy'],
 ['anger', 'surprise', 'joy'],
 ['anger'],
 ['anger', 'disgust'],
 ['disgust', 'sadness'],
 ['anger'],
 ['fear', 'surprise'],
 ['surprise'],
 ['joy'],
 ['anger', 'surprise'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['anger', 'surprise'],
 ['anger', 'surprise'],
 ['joy'],
 ['anger', 'joy'],
 ['anger', 'fear'],
 ['surprise'],
 ['anger', 'disgust'],
 ['sadness', 'respect'],
 ['fear', 'sadness'],
 ['neutral'],
 ['neutral'],
 ['disgust', 'sadness'],
 ['surprise', 'anger'],
 ['anger', 'surprise'],
 ['disgust'],
 ['anger'],
 ['disgust', 'fear'],
 ['anger', 'disgust'],
 ['anger', 'sadness'],
 ['joy'],
 ['neutral'],
 ['joy'],
 ['joy'],
 ['anger', 'disgust'],
 ['anger', 'surprise'],
 ['anger', 'joy'],
 ['joy'

In [76]:
grounds = test_df.emotions_list.tolist()

In [77]:
len(grounds)

1776

In [78]:
grounds

[['surprise', 'joy'],
 ['joy'],
 ['surprise', 'joy'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['surprise'],
 ['joy'],
 ['joy'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['neutral'],
 ['sadness'],
 ['sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['fear', 'surprise'],
 ['surprise'],
 ['joy'],
 ['anger', 'surprise'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['surprise', 'joy'],
 ['fear', 'sadness'],
 ['fear', 'sadness'],
 ['fear', 'surprise'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['fear', 'sadness'],
 ['fear', 'sadness', 'surprise'],
 ['sadness'],
 ['sadness'],
 ['fear', 'sadness'],
 ['sadness', 'surprise'],
 ['sadness', 'surprise'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['anger', 'disgust'],
 ['joy'],
 ['joy'],
 ['surprise', 'joy'],
 ['surprise', 'joy'],
 ['anger', 'surprise'],
 ['anger', 'surprise'],
 ['neutral'],
 ['joy'],
 ['joy'],
 ['neutral'

In [79]:
len(grounds)

1776

In [80]:
all_labels = ["anger", "surprise", "fear", "disgust", "sadness", "joy", "neutral"]

def labels_to_binary_matrix(label_list, all_labels):
    binary_matrix = np.zeros((len(label_list), len(all_labels)))
    for i, labels in enumerate(label_list):
        for label in labels:
            if label in all_labels:
                binary_matrix[i][all_labels.index(label)] = 1
    return binary_matrix

def opposite(component_type):

    if component_type == "anger":
        return "surprise"
    elif component_type == "disgust":
        return "joy"
    elif component_type == "fear":
        return "sadness"
    elif component_type == "sadness":
        return "anger"
    elif component_type == "surprise":
        return "disgust"
    elif component_type == "joy":
        return "fear"
    elif component_type == "Neutral":
        return "sadness"
    

def harmonize_preds(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds 

def post_process_zs(grounds, preds):

    for i,(x,y) in enumerate(zip(grounds, preds)):
        
        if len(x) != len(y):
            
            preds[i] = harmonize_preds(x, y)

    true_matrix = labels_to_binary_matrix(grounds, all_labels)
    predicted_matrix = labels_to_binary_matrix(preds, all_labels)

    return true_matrix, predicted_matrix

In [81]:
true_matrix, predicted_matrix = post_process_zs(grounds, preds)

In [82]:
print(classification_report(true_matrix, predicted_matrix, target_names=all_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.594     0.567     0.580       614
    surprise      0.632     0.597     0.614       486
        fear      0.474     0.410     0.440       407
     disgust      0.152     0.400     0.220        85
     sadness      0.492     0.337     0.400       347
         joy      0.564     0.592     0.578       429
     neutral      0.216     0.225     0.221       129

   micro avg      0.507     0.496     0.502      2497
   macro avg      0.446     0.447     0.436      2497
weighted avg      0.528     0.496     0.508      2497
 samples avg      0.494     0.486     0.489      2497



/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [44]:
df

,file_name,page_nr,panel_nr,balloon_nr,utterance,raw_annotation,raw_emotion,raw_speaker_id,emotion,speaker_id,split,emotions_list
0,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,1,DID YOU HAVE TO ELECTROCUTE HER SO HARD?,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN0-DI0-FE3-SA0-SU5-JO0,ID-1,TRAIN,"[fear, surprise]"
1,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,2,IT'S NOT LIKE I HAVE DIFFERENT SETTINGS.,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-2,AN0-DI0-FE0-SA0-SU5-JO0,ID-2,TRAIN,[surprise]
2,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,3,YOU'RE ELECTROCUTIONER. IT'S YOUR WHOLE THING....,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN0-DI0-FE2-SA0-SU0-JO0,ID-1,TRAIN,[fear]
3,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,3,1,"OH, HEY. I THINK SHE'S AWAKE.",2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-2,AN0-DI0-FE0-SA0-SU4-JO0,ID-2,TRAIN,[surprise]
4,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,4,1,"WELCOME BACK, MADAM MAYOR. BLOCKBUSTER IS PRET...",2024-08-27 - aselermekova20\nFeeling:AN3-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN3-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN3-DI0-FE0-SA0-SU0-JO0,ID-1,TRAIN,[anger]
...,...,...,...,...,...,...,...,...,...,...,...,...
5277,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,20,1,1,I KNOW THE BEINGS OF THIS WORLD ARE TRYING TO ...,2024-08-27 - aselermekova20\nFeeling:AN5-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN5-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:BLACKMA...,AN5-DI0-FE0-SA0-SU0-JO0,BLACKMANTASAURUS,TEST,[anger]
5278,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,20,1,2,… BUT I WILL CRUSH THEM IN DUE TIME!,2024-08-27 - aselermekova20\nFeeling:AN5-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN5-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:BLACKMA...,AN5-DI0-FE0-SA0-SU0-JO0,BLACKMANTASAURUS,TEST,[anger]
5279,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,20,2,1,FOR MY FIRST TASK...,2024-08-27 - aselermekova20\nFeeling:AN5-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN5-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:BLACKMA...,AN5-DI0-FE0-SA0-SU0-JO0,BLACKMANTASAURUS,TEST,[anger]
5280,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,20,2,2,… I MUST REMOVE THIS WORLD OF THEIR GODS!,2024-08-27 - aselermekova20\nFeeling:AN5-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN5-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:BLACKMA...,AN5-DI0-FE0-SA0-SU0-JO5,BLACKMANTASAURUS,TEST,"[anger, joy]"


In [45]:
df.emotions_list.value_counts()

emotions_list
[anger]                                793
[joy]                                  714
[sadness]                              467
[surprise]                             436
[fear, surprise]                       400
[neutral]                              343
[fear]                                 335
[fear, sadness]                        237
[anger, surprise]                      215
[surprise, joy]                        198
[anger, sadness]                       178
[anger, fear]                          164
[anger, disgust]                       142
[sadness, surprise]                    128
[anger, fear, surprise]                 85
[anger, joy]                            72
[sadness, joy]                          53
[anger, fear, sadness]                  46
[fear, sadness, surprise]               41
[disgust, surprise]                     30
[anger, disgust, sadness]               26
[disgust, sadness]                      26
[fear, joy]                             

In [53]:
df['word_count'] = df['utterance'].apply(lambda x: len(str(x).split()))

# Convert lists in 'emotions_list' column to tuples (to make them hashable)
df['emotions_list_tuple'] = df['emotions_list'].apply(lambda x: tuple(x))

# Group by the 'emotions_list_tuple' column and calculate the average word count
average_word_count_by_emotion = df.groupby('emotions_list_tuple')['word_count'].mean()

In [54]:
average_word_count_by_emotion

emotions_list_tuple
(anger,)                                8.005044
(anger, disgust)                       10.169014
(anger, disgust, fear)                  7.538462
(anger, disgust, fear, sadness)        15.500000
(anger, disgust, fear, surprise)        3.000000
(anger, disgust, joy)                  13.500000
(anger, disgust, sadness)              15.730769
(anger, disgust, sadness, surprise)    11.000000
(anger, disgust, surprise)             10.722222
(anger, fear)                           9.018293
(anger, fear, joy)                     15.333333
(anger, fear, sadness)                  9.847826
(anger, fear, sadness, joy)             1.000000
(anger, fear, sadness, surprise)       21.000000
(anger, fear, surprise)                 8.188235
(anger, joy)                            9.569444
(anger, sadness)                       11.112360
(anger, sadness, surprise)             10.722222
(anger, sadness, surprise, joy)         8.000000
(anger, surprise)                       8.893023
